In [1]:
# Hahah :)
# https://github.com/H-Elbez/Nengo
# Let's see if that does ANYTHING

In [2]:
# Let's just rebuild the entire thing,
# I kinda don't want to mess with the other version

# Don't be nervous about doing wrong things in here! 
# See it as a draft version, or a messing around version, you can make a clean version of this later
# These are safe testing grounds where we mess around with a lot of stuff

In [1]:
# Imports
import nengo
import nengo_dl
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from skimage.measure import block_reduce

from STDP_learning import STDP
from DataLog import DataLog

In [2]:
def checkShape(var, kernel):
    '''Check shapes for convolution
    Args:
        var (ndarray): 2d or 3d input array for convolution.
        kernel (ndarray): 2d or 3d convolution kernel.
    Returns:
        kernel (ndarray): 2d kernel reshape into 3d if needed.
    '''
    var_ndim = np.ndim(var)
    kernel_ndim = np.ndim(kernel)
    if var_ndim not in [2, 3]:
        raise Exception("<var> dimension should be in 2 or 3.")
    if kernel_ndim not in [2, 3]:
        raise Exception("<kernel> dimension should be in 2 or 3.")
    if var_ndim < kernel_ndim:
        raise Exception("<kernel> dimension > <var>.")
    if var_ndim == 3 and kernel_ndim == 2:
        kernel = np.repeat(kernel[:, :, None], var.shape[2], axis=2)
    return kernel

def asStride(arr, sub_shape, stride):
    '''Get a strided sub-matrices view of an ndarray.
    Args:
        arr (ndarray): input array of rank 2 or 3, with shape (m1, n1) or (m1, n1, c).
        sub_shape (tuple): window size: (m2, n2).
        stride (int): stride of windows in both y- and x- dimensions.
    Returns:
        subs (view): strided window view.
    See also skimage.util.shape.view_as_windows()
    '''
    s0, s1 = arr.strides[:2]
    m1, n1 = arr.shape[:2]
    m2, n2 = sub_shape[:2]
    view_shape = (1+(m1-m2)//stride, 1+(n1-n2)//stride, m2, n2)+arr.shape[2:]
    strides = (stride*s0, stride*s1, s0, s1)+arr.strides[2:]
    subs = np.lib.stride_tricks.as_strided(
        arr, view_shape, strides=strides, writeable=False)
    return subs

def conv3D3(var, kernel, stride=1, pad=0):
    '''3D convolution by strided view.
    Args:
        var (ndarray): 2d or 3d array to convolve along the first 2 dimensions.
        kernel (ndarray): 2d or 3d kernel to convolve. If <var> is 3d and <kernel>
            is 2d, create a dummy dimension to be the 3rd dimension in kernel.
    Keyword Args:
        stride (int): stride along the 1st 2 dimensions. Default to 1.
        pad (int): number of columns/rows to pad at edges.
    Returns:
        conv (ndarray): convolution result.
    '''
    kernel = checkShape(var, kernel)
    if pad > 0:
        var_pad = padArray(var, pad, pad)
    else:
        var_pad = var
    view = asStride(var_pad, kernel.shape, stride)
    if np.ndim(kernel) == 2:
        conv = np.sum(view*kernel, axis=(2, 3))
    else:
        conv = np.sum(view*kernel, axis=(2, 3, 4))
    return conv

In [36]:
# Parameters from the paper
n_freqbands = 40
n_timeframes = 41
n_timesteps = 30

n_neurons = n_freqbands * n_timeframes * n_timesteps

n_samples = 2464

f_maps = 50

window_size = [6,n_freqbands]
conv_inp_shape = (n_freqbands, n_timeframes, n_timesteps)
stride = [1,1]

mean = 0.8
std = 0.05

LIMIT_TEST = 250

presentation_time = 0.001

threshold = 23
thresh_config = nengo.presets.ThresholdingEnsembles(threshold) # Set the threshold

In [37]:
# Loading data
train = np.load("spike_trains_train_fixed2.npy")

# [n_freqbands* n_timeframes * n_timesteps, n_samples] or in other words [data, n_samples]
# train = np.reshape(train, (n_freqbands*n_timeframes*n_timesteps, n_samples))

# train = train[:,0:LIMIT_TEST]

# train.shape

In [38]:
train_conv = np.reshape(train, (40,41,30,2464))
train_new = []

for n in range(n_samples):
    train_new.append(conv3D3(train_conv[:,:,:,n], np.array([[40,6]])))
    
train_new = np.array(train_new)

In [39]:
train_new.T.shape

(40, 40, 2464)

In [40]:
train_new = np.reshape(train_new.T, (1600,2464))[:,0:LIMIT_TEST]

In [41]:
%%time

model = nengo.Network(label="Audio STDP learning")

# Log and reduce collected Probes to avoid getting memory issues
full_log = False

if(not full_log):
    log = DataLog()

with model:

    # Layers ----------------------------------------------------------------------
    input_layer = nengo.Node(nengo.processes.PresentInput(train_new, presentation_time))
    
    pre = nengo.Ensemble(LIMIT_TEST, dimensions=LIMIT_TEST)
    
    post = nengo.Ensemble(LIMIT_TEST, dimensions=LIMIT_TEST)
    
    # Transformations -------------------------------------------------------------
#     transform = nengo.Convolution(
#                 n_filters = f_maps,
#                 input_shape = conv_inp_shape,
#                 kernel_size = window_size,
#                 strides = stride,
#                 padding="valid", # previously: "same"
#                 channels_last = True,
#                 init = nengo.dists.Gaussian(mean, std)
#             )
    
    # Connections -----------------------------------------------------------------
    
#     conv_conn = nengo.Connection(input_layer, pre, transform = transform)
    
    learn_conn = nengo.Connection(
        pre, post,
        learning_rule_type = nengo.BCM(learning_rate=5e-10), # Change this later
        solver = nengo.solvers.LstsqL2(weights=True)
    )
    
    # Probes ----------------------------------------------------------------------
#     input_probe = nengo.Probe(input_layer)
    
#     pre_probe = nengo.Probe(pre, synapse=0.01) # Do these make sense...?
#     post_probe = nengo.Probe(post, synapse=0.01)
    
    layer1_synapses_probe = nengo.Probe(learn_conn,"weights",label="layer1_synapses") # This seems to be a very big one, that those other people make the heatmap from
    
    if(not full_log):
        nengo.Node(log)
    
print("Running for {} seconds".format(len(train)*presentation_time))
with nengo_dl.Simulator(model) as sim:
        
    sim.run(len(train)*presentation_time) # This makes sense right, since you're now presenting every point for 0.1 second


Running for 2.464 seconds
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Simulation finished in 0:00:14                                                 
CPU times: total: 22.7 s
Wall time: 15.6 s


In [42]:
%%time

sim.data[layer1_synapses_probe].shape

# sum(sim.data[layer1_synapses_probe][:,0,:])

# with open('weights.npy', 'wb') as f:
#     np.save(f, sim.data[layer1_synapses_probe])

CPU times: total: 453 ms
Wall time: 456 ms


(2464, 250, 250)

In [44]:
sample_output = sim.data[layer1_synapses_probe]

avg = sum(sample_output)/sample_output.shape[1]

avg.shape

(250, 250)

In [46]:
pooled = block_reduce(avg, (4), np.max)

pooled

array([[0.10675056, 0.08300735, 0.1355629 , ..., 0.08745863, 0.00063955,
        0.13214238],
       [0.10887646, 0.08229525, 0.1413293 , ..., 0.08850267, 0.00482082,
        0.13497776],
       [0.1107283 , 0.08471756, 0.14173488, ..., 0.10700819, 0.00025525,
        0.1321756 ],
       ...,
       [0.10842292, 0.09115589, 0.14779562, ..., 0.1199377 , 0.00371732,
        0.14109948],
       [0.10463195, 0.08498006, 0.13435507, ..., 0.08810836, 0.00025392,
        0.12172994],
       [0.09835338, 0.07705263, 0.12826617, ..., 0.08532179, 0.04395097,
        0.1198554 ]], dtype=float32)

In [8]:
for x in range(50):
    print()

In [ ]:
# This is a silly thing I do so the notebook let's me scroll down :p